In [76]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import torch.nn.functional as F
import torch.optim as optim
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff

In [3]:
# Data preprocessing
drone = pd.read_json('../data/data_20092023-15:36:05(more_movement).json')
gcs = pd.read_json('../data/data_20092023-15:39:26(less_movement).json')

# add prefix to packet type
drone['mavpackettype'] = 'UAV_' + drone['mavpackettype']
gcs['mavpackettype'] = 'GCS_' + gcs['mavpackettype']

# zero time to the beginning of the flight
drone['time_boot_ms'] -= drone['time_boot_ms'].min()
gcs['time_boot_ms'] -= gcs['time_boot_ms'].min()

# combine dataframes
df = pd.concat([drone, gcs], ignore_index=True)
df = df.sort_values(by=['time_boot_ms'], ignore_index=True)

df['time_s'] = df['time_boot_ms'] / 1000
df['lat'] = df['lat'] / 1e7 # int->float
df['lon'] = df['lon'] / 1e7
df['alt'] = df['alt'] / 1000 # orginally in mm

df['vx'] = df['vx'] / 100 # to m/s
df['vy'] = df['vy'] / 100
df['vz'] = -1*(df['vz'] / 100) # Invert z velocity. Up is now positive.

df = df[
    df['mavpackettype'].str.contains('UAV_GLOBAL_POSITION_INT')
]
df = df[['time_s', 'lat', 'lon', 'alt', 'vx', 'vy', 'vz']]
df.set_index('time_s', inplace=True)
df.index.name = 'time'
df

,lat,lon,alt,vx,vy,vz
time,,,,,,
0.140,42.724676,-84.480802,274.12,-0.01,-0.02,-0.00
0.473,42.724676,-84.480802,274.12,0.00,-0.02,-0.00
0.806,42.724676,-84.480802,274.12,0.00,-0.02,-0.00
1.138,42.724676,-84.480802,274.11,0.00,-0.02,-0.00
1.471,42.724675,-84.480802,274.11,0.00,-0.02,-0.00
...,...,...,...,...,...,...
75.398,42.724638,-84.480762,275.50,1.22,-0.39,-0.20
75.731,42.724641,-84.480763,275.55,0.92,-0.65,-0.00
76.063,42.724644,-84.480766,275.55,0.89,-0.89,-0.01


In [4]:
import pymap3d as pm

def geodesic_distances(lat1, lon1, alt1, lat2, lon2, alt2):
    # In mavlink, east is y, north is x, down is z (z flipped upwards in preprocessing)
    y, x, z = pm.geodetic2enu(lat1, lon1, alt1, lat2, lon2, alt2)
    return x, y, z

origin = df.iloc[0]

df['x'], df['y'], df['z'] = geodesic_distances(
    df['lat'], df['lon'], df['alt'], origin['lat'], origin['lon'], origin['alt']
)

df

,lat,lon,alt,vx,vy,vz,x,y,z
time,,,,,,,,,
0.140,42.724676,-84.480802,274.12,-0.01,-0.02,-0.00,0.000000,0.000000,0.000000e+00
0.473,42.724676,-84.480802,274.12,0.00,-0.02,-0.00,-0.011109,-0.008191,-2.737780e-10
0.806,42.724676,-84.480802,274.12,0.00,-0.02,-0.00,-0.022218,-0.016382,6.995199e-10
1.138,42.724676,-84.480802,274.11,0.00,-0.02,-0.00,-0.033328,-0.032763,-9.999998e-03
1.471,42.724675,-84.480802,274.11,0.00,-0.02,-0.00,-0.044437,-0.040954,-9.999999e-03
...,...,...,...,...,...,...,...,...,...
75.398,42.724638,-84.480762,275.50,1.22,-0.39,-0.20,-4.154847,3.276307,1.379998e+00
75.731,42.724641,-84.480763,275.55,0.92,-0.65,-0.00,-3.821570,3.137064,1.429998e+00
76.063,42.724644,-84.480766,275.55,0.89,-0.89,-0.01,-3.588277,2.907722,1.429998e+00


In [7]:
def moving_average(x, n=3):
    return np.convolve(x, np.ones(n), 'same') / n

avg_flag = True

if avg_flag:

    # Take moving average of input data:

    n = 4

    df['x'] = moving_average(df['x'], n)
    df['y'] = moving_average(df['y'], n)
    df['z'] = moving_average(df['z'], n)

df

,lat,lon,alt,vx,vy,vz,x,y,z
time,,,,,,,,,
0.140,42.724676,-84.480802,274.12,-0.01,-0.02,-0.00,-0.002777,-0.002048,-6.844449e-11
0.473,42.724676,-84.480802,274.12,0.00,-0.02,-0.00,-0.008332,-0.006143,1.064355e-10
0.806,42.724676,-84.480802,274.12,0.00,-0.02,-0.00,-0.016664,-0.014334,-2.499999e-03
1.138,42.724676,-84.480802,274.11,0.00,-0.02,-0.00,-0.027773,-0.024572,-4.999999e-03
1.471,42.724675,-84.480802,274.11,0.00,-0.02,-0.00,-0.038882,-0.034811,-4.999999e-03
...,...,...,...,...,...,...,...,...,...
75.398,42.724638,-84.480762,275.50,1.22,-0.39,-0.20,-4.354813,3.307023,1.384998e+00
75.731,42.724641,-84.480763,275.55,0.92,-0.65,-0.00,-4.024314,3.167779,1.397498e+00
76.063,42.724644,-84.480766,275.55,0.89,-0.89,-0.01,-3.732697,2.985535,1.429998e+00


In [13]:
df['next_x'] = df['x'].shift(-1)
df['next_y'] = df['y'].shift(-1)
df['next_z'] = df['z'].shift(-1)

In [14]:
df.iloc[:-1]

,lat,lon,alt,vx,vy,vz,x,y,z,next_x,next_y,next_z
time,,,,,,,,,,,,
0.140,42.724676,-84.480802,274.12,-0.01,-0.02,-0.00,-0.002777,-0.002048,-6.844449e-11,-0.008332,-0.006143,1.064355e-10
0.473,42.724676,-84.480802,274.12,0.00,-0.02,-0.00,-0.008332,-0.006143,1.064355e-10,-0.016664,-0.014334,-2.499999e-03
0.806,42.724676,-84.480802,274.12,0.00,-0.02,-0.00,-0.016664,-0.014334,-2.499999e-03,-0.027773,-0.024572,-4.999999e-03
1.138,42.724676,-84.480802,274.11,0.00,-0.02,-0.00,-0.027773,-0.024572,-4.999999e-03,-0.038882,-0.034811,-4.999999e-03
1.471,42.724675,-84.480802,274.11,0.00,-0.02,-0.00,-0.038882,-0.034811,-4.999999e-03,-0.049991,-0.045049,-2.499999e-03
...,...,...,...,...,...,...,...,...,...,...,...,...
75.066,42.724635,-84.480761,275.47,1.14,-0.36,0.08,-4.721417,3.395073,1.369997e+00,-4.354813,3.307023,1.384998e+00
75.398,42.724638,-84.480762,275.50,1.22,-0.39,-0.20,-4.354813,3.307023,1.384998e+00,-4.024314,3.167779,1.397498e+00
75.731,42.724641,-84.480763,275.55,0.92,-0.65,-0.00,-4.024314,3.167779,1.397498e+00,-3.732697,2.985535,1.429998e+00


In [104]:
class DNN(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.ln1 = torch.nn.Linear(input_size, 5, dtype=torch.float32)
        self.ln2 = torch.nn.Linear(5, output_size, dtype=torch.float32)
    def forward(self, x):
        x = F.relu(self.ln1(x))
        x = self.ln2(x)
        return x

In [12]:
from sklearn.model_selection import train_test_split

In [64]:
X = torch.tensor(df.iloc[:-1][['vx', 'vy', 'vz', 'x', 'y', 'z']].values, dtype=torch.float32)
y = torch.tensor(df.iloc[:-1][['next_x', 'next_y', 'next_z']].values, dtype=torch.float32)

In [65]:
y

tensor([[-8.3319e-03, -6.1431e-03,  1.0644e-10],
        [-1.6664e-02, -1.4334e-02, -2.5000e-03],
        [-2.7773e-02, -2.4572e-02, -5.0000e-03],
        [-3.8882e-02, -3.4811e-02, -5.0000e-03],
        [-4.9991e-02, -4.5049e-02, -2.5000e-03],
        [-6.1101e-02, -5.3240e-02,  2.5000e-03],
        [-7.4987e-02, -6.1431e-02,  2.5000e-03],
        [-8.6096e-02, -7.1669e-02, -2.5000e-03],
        [-1.0276e-01, -8.1908e-02, -1.2500e-02],
        [-1.1942e-01, -9.2146e-02, -2.5000e-02],
        [-1.3053e-01, -1.0238e-01, -3.2500e-02],
        [-1.4720e-01, -1.1262e-01, -4.0000e-02],
        [-1.5831e-01, -1.2286e-01, -4.5000e-02],
        [-1.7219e-01, -1.3310e-01, -5.0000e-02],
        [-1.9163e-01, -1.4334e-01, -5.5000e-02],
        [-2.1385e-01, -1.5153e-01, -5.7500e-02],
        [-2.4163e-01, -1.5767e-01, -6.0000e-02],
        [-2.6662e-01, -1.6586e-01, -5.2500e-02],
        [-2.8328e-01, -1.8224e-01, -8.7333e-09],
        [-2.8884e-01, -2.0067e-01,  1.3500e-01],
        [-2.7218e-01

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

In [108]:
model = DNN(6, 3)
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-1)

In [109]:

def train_model(model):
    
    patience = 0

    model.train()
    kf = KFold(n_splits=5)
    patience = 0
    best_loss = 5 * [100000]
    while True:
        for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
            print(f'Fold {fold+1}')

            X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
            y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
            for epoch in range(10):  
                    optimizer.zero_grad()
                    
                    outputs = model(X_train_fold)
                    loss = criterion(outputs, y_train_fold)
                    loss.backward()
                    optimizer.step()
                    
                    # Validation step, torch.no_grad() prevents the model from running the optimizer
                    # and updating its weights. Validation is used so that the model is not overfit on 
                    # training data. Acts as a testing data set sort of
                    with torch.no_grad():
                        val_outputs = model(X_val_fold)
                        val_loss = criterion(val_outputs, y_val_fold)

                    print(f'[{epoch + 1}] Training loss: {loss.item():.5f}, Validation loss: {val_loss.item():.5f}')
                    if val_loss.item() < best_loss[fold]:
                        best_loss[fold] = val_loss.item()
                        patience = 0
                    elif val_loss.item() > best_loss[fold]: 
                        patience += 1
                        if patience == 50:
                            print(f'Finished training with best val loss of {best_loss}')
                            return

            print('Finished Training for Fold', fold+1)

In [110]:
train_model(model)

Fold 1
[1] Training loss: 19.25505, Validation loss: 6.85284
[2] Training loss: 10.52119, Validation loss: 4.07500
[3] Training loss: 6.47757, Validation loss: 3.71373
[4] Training loss: 4.89109, Validation loss: 4.88943
[5] Training loss: 5.25455, Validation loss: 5.60179
[6] Training loss: 5.43694, Validation loss: 4.72831
[7] Training loss: 4.51184, Validation loss: 2.62121
[8] Training loss: 2.61811, Validation loss: 1.28533
[9] Training loss: 1.46424, Validation loss: 1.87631
[10] Training loss: 2.45416, Validation loss: 2.58092
Finished Training for Fold 1
Fold 2
[1] Training loss: 3.35465, Validation loss: 1.93298
[2] Training loss: 2.49047, Validation loss: 1.25155
[3] Training loss: 1.45820, Validation loss: 1.14939
[4] Training loss: 1.20018, Validation loss: 1.31747
[5] Training loss: 1.40810, Validation loss: 1.37148
[6] Training loss: 1.53092, Validation loss: 1.13622
[7] Training loss: 1.37430, Validation loss: 0.86587
[8] Training loss: 1.15223, Validation loss: 0.77758


In [92]:
model.eval()
model(torch.tensor([-0.01	,-0.02	,-0.00,-0.002777,	-0.002048	,-6.741632e-11]))

tensor([-0.0220, -0.0006, -0.0048], grad_fn=<ViewBackward0>)

In [111]:
preds = model(X_test)

In [112]:
error = 100 * np.sqrt(
    (preds[:,0].detach().numpy()- y_test[:,0].detach().numpy())**2+
    (preds[:,1].detach().numpy() - y_test[:,1].detach().numpy())**2+
    (preds[:,2].detach().numpy() - y_test[:,2].detach().numpy())**2
)

In [113]:
error

array([ 7.335676 ,  3.80735  ,  2.8104038,  6.2649884,  1.901505 ,
        8.850325 ,  9.392192 ,  2.468662 , 10.172497 ,  1.9837635,
        5.4393625,  9.400205 ,  3.9171576,  2.4454427,  3.6539502,
        2.608984 ,  5.42445  ,  3.1293783,  4.7782617,  2.7486491,
        7.178949 ,  5.858509 ,  5.1245847,  6.1334944,  1.7875158,
        4.0902896, 10.657119 ,  4.5282993,  5.736521 ,  0.9094934,
        7.390444 ,  2.6743143,  3.8334715,  2.950456 ,  1.9520621,
        1.2546877,  9.109951 ,  2.790603 ,  3.562508 ,  5.201876 ,
        6.860573 ,  9.765763 ,  6.7669907,  5.4973884,  7.554003 ,
        6.2445173, 10.459678 ,  5.550785 ,  7.0433154,  6.9319963,
       11.077743 ,  8.714457 ,  3.9148164,  7.342495 ,  3.7699673,
        5.906522 ,  3.5928612,  5.751227 ,  4.7537403, 10.379705 ,
        3.0343637,  4.892866 ,  5.557143 ,  2.3704443,  8.643344 ,
        6.3986936,  6.932155 ,  2.251078 ,  8.73214  ], dtype=float32)

In [114]:
fig = px.histogram(error, nbins=50, marginal="violin") # or violin,)
fig.update_layout(
    title_text='Error in Position Prediction',
    xaxis_title_text='Error (cm)',
    yaxis_title_text='Count',
    bargap=0.2,
    bargroupgap=0.1
)

In [116]:
ninefifthpercent = np.quantile(error, 0.95)
print(f'95% of the error is less than {ninefifthpercent:.2f} cm')

95% of the error is less than 10.30 cm


In [117]:
y_pred_train = model(X_train)

In [119]:
training_error = 100 * np.sqrt(
    (y_pred_train[:,0].detach().numpy()- y_train[:,0].detach().numpy())**2+
    (y_pred_train[:,1].detach().numpy() - y_train[:,1].detach().numpy())**2+
    (y_pred_train[:,2].detach().numpy() - y_train[:,2].detach().numpy())**2
)

In [120]:
fig = px.histogram(training_error, nbins=50, marginal="violin") # or violin,)
fig.update_layout(
    title_text='Error in Position Prediction',
    xaxis_title_text='Error (cm)',
    yaxis_title_text='Count',
    bargap=0.2,
    bargroupgap=0.1
)

In [121]:
ninefifthpercent = np.quantile(training_error, 0.95)
print(f'95% of the error is less than {ninefifthpercent:.2f} cm')

95% of the error is less than 10.31 cm


In [123]:
av = training_error.mean()
print(f'Average error is {av:.2f} cm')

Average error is 5.89 cm


In [124]:
sd = training_error.std()
print(f'Standard Deviation is: {sd:.2f} cm')

Standard Deviation is: 7.39 cm
